In [1]:
import pandas as pd
import random
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import time
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
import gc

In [2]:
file_path = '../../data/input/integrated_data_dummy.csv'

data = pd.read_csv(file_path)
data = data.sort_values(["busCode","busCodeSB"])

In [3]:
# data.select_dtypes(['int32'])
for col in data:
    if data[col].dtype == 'int64':
        data[col] = data[col].astype('float64', copy=False)

route
tripNum
shapeSequence
busCode
stopPointId
alertConfidence
alertReliability
alertIsJamUnifiedAlert
alertInScale
jamDelay
jamLength
jamLevel
jamSeverity
headway
headwayThreshold
busBunching
GPShour
tripNumSB
shapeSequenceSB
busCodeSB
stopPointIdSB
alertConfidenceSB
alertReliabilitySB
alertIsJamUnifiedAlertSB
alertInScaleSB
jamDelaySB
jamLengthSB
jamLevelSB
jamSeveritySB
DAY(gps_datetime)
DAY(precipitationTime)
DAY(alertDateTime)
DAY(jamUpdateDateTime)
DAY(gps_datetimeSB)
DAY(precipitationTimeSB)
DAY(alertDateTimeSB)
DAY(jamUpdateDateTimeSB)
YEAR(gps_datetime)
YEAR(precipitationTime)
YEAR(alertDateTime)
YEAR(jamUpdateDateTime)
YEAR(gps_datetimeSB)
YEAR(precipitationTimeSB)
YEAR(alertDateTimeSB)
YEAR(jamUpdateDateTimeSB)
MONTH(gps_datetime)
MONTH(precipitationTime)
MONTH(alertDateTime)
MONTH(jamUpdateDateTime)
MONTH(gps_datetimeSB)
MONTH(precipitationTimeSB)
MONTH(alertDateTimeSB)
MONTH(jamUpdateDateTimeSB)
WEEKDAY(gps_datetime)
WEEKDAY(precipitationTime)
WEEKDAY(alertDateTime)
WEEKD

In [4]:
data.shapeSequence.dtype

dtype('float64')

In [5]:
# Get label column and remove it from data
y = data['busBunching']
# y_threshold = data['headwayThreshold']

cols = ['headway', 'busBunching', "headwayThreshold"]
data.drop(columns=cols, inplace=True)

In [ ]:
# Normalize data

cols_len = len(data.columns)

for c in range(cols_len):
    col_values = data.iloc[:,c]
    min_col = min(col_values)
    max_col = max(col_values)
    
    col_values_len = len(col_values)-1
#     print(col_values)
#     print("Min " + str(min_col) + " Max " + str(max_col))
    for i in range(col_values_len):
#         print(col_values[i+1])
        
        if(min_col == max_col):
            data.iat[i,c] = 0.0
        else:
            data.iat[i,c] = ((col_values[i+1] - min_col) / (max_col - min_col))
            
#         print(test.iloc[i,c])

In [ ]:
output_path = '../../data/output/normalized_data_X.csv'

data.to_csv(output_path)

In [ ]:
data

In [8]:
data.isnull().values.any()

False

In [9]:
data.columns[data.isnull().any()]

Index([], dtype='object')

In [ ]:
# Normalize data
min_max_scaler = preprocessing.MinMaxScaler()
data_scale = min_max_scaler.fit_transform(data)

In [10]:
# Making training and test data: 80% Training, 20% Test
random.seed(15) #to get always the same set
train_X, test_X, train_Y, test_Y = train_test_split(data, y, test_size=0.20, random_state=7)

In [11]:
del y
del data

In [13]:
def rmse_cv(model, X_train, y_train):
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring = "neg_mean_squared_error", cv = 5))
    return(rmse)

# function to plot the RMSE vs parameter value
def plot_rmse_param(series, param_name):
    series.plot(title = "Validation Error vs " + param_name)
    plt.xlabel(param_name)
    plt.ylabel("RMSE")
    
# function to get the best RMSE and the best parameter value of the model
def best_rmse_param(series):
    best_rmse = series.min()
    best_param = series.idxmin() 
    
    return(best_rmse, best_param)

In [ ]:
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel, random_state=0, n_jobs=-1)

gpc.fit(train_X, train_Y)